In [1]:
# !conda activate final_project_env
# !mkdir genome
# %cd genome
# !wget -nc -O genome/genome.fa.gz https://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS19/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS19.genomic.fa.gz 
# !gzip -d -f genome/genome.fa.gz

--2024-12-05 14:05:07--  https://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS19/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS19.genomic.fa.gz
Resolving ftp.ebi.ac.uk... 193.62.193.165
Connecting to ftp.ebi.ac.uk|193.62.193.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116797085 (111M) [application/x-gzip]
Saving to: ‘genome/genome.fa.gz’

genome/genome.fa.gz 100%[===================>] 111.39M   590KB/s    in 3m 14s  

2024-12-05 14:08:22 (589 KB/s) - ‘genome/genome.fa.gz’ saved [116797085/116797085]



In [2]:
# !wget -O - https://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS19/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS19.canonical_geneset.gtf.gz | \
#     gzip -f -d > genome/annotations.gtf

--2024-11-21 16:30:04--  https://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS19/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS19.canonical_geneset.gtf.gz
Resolving ftp.ebi.ac.uk... 193.62.193.165
Connecting to ftp.ebi.ac.uk|193.62.193.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3228084 (3.1M) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>]   3.08M  3.82MB/s    in 0.8s    

2024-11-21 16:30:06 (3.82 MB/s) - written to stdout [3228084/3228084]



In [3]:
# !samtools faidx genome/genome.fa

In [4]:
# !mkdir fastq_trimmed
# # !cut  -d ' ' -f 2 /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/M007087_WheelerN_md5sum.txt | tail -n +2 > fastq_files.txt
# # !while IFS= read -r line; do \
#  
# #     done < fastq_files.txt
# #  -q 37 

mkdir: cannot create directory ‘fastq_trimmed’: File exists


In [30]:
for i in ["Int","Liv"]:
    for a in  ["01","02","03","04"]:
        print(i + "_" + a)
        print(i + "_" + a)

    

Int_01
Int_01
Int_02
Int_02
Int_03
Int_03
Int_04
Int_04
Liv_01
Liv_01
Liv_02
Liv_02
Liv_03
Liv_03
Liv_04
Liv_04


In [17]:
# !for fastq in /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/*_R1_100.fastq.gz; do \
#     for fastq2 in /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/*_R2_100.fastq.gz; do \
#         base_name=$(basename "$fastq" .fastq.gz); \
#     cutadapt -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -o ./fastq_trimmed/$base_name.fastq.gz $fastq $fastq2; \
# done

This is cutadapt 4.9 with Python 3.12.3
Command line parameters: -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -o ./fastq_trimmed/*_R1_100.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/*_R1_100.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/*_R2_100.fastq.gz
When a paired-end trimming option such as -A/-G/-B/-U, is used, a second output file needs to be specified via -p (--paired-output).


In [10]:
# !cutadapt -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -o ./fastq_trimmed/Int_01_1.fastq.gz -p ./fastq_trimmed/Int_01_2.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/Int-01_S42_L005_R1_001.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/Int-01_S42_L005_R2_001.fastq.gz
# !cutadapt -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -o ./fastq_trimmed/Int_02_1.fastq.gz -p ./fastq_trimmed/Int_02_2.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/Int-02_S43_L005_R1_001.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/Int-02_S43_L005_R2_001.fastq.gz
# !cutadapt -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -o ./fastq_trimmed/Int_03_1.fastq.gz -p ./fastq_trimmed/Int_03_2.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/Int-03_S44_L005_R1_001.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/Int-03_S44_L005_R2_001.fastq.gz
!cutadapt -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -o ./fastq_trimmed/Int_04_1.fastq.gz -p ./fastq_trimmed/Int_04_2.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/Int-04_S45_L005_R1_001.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/Int-04_S45_L005_R2_001.fastq.gz
!cutadapt -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -o ./fastq_trimmed/Liv_01_1.fastq.gz -p ./fastq_trimmed/Liv_01_2.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/Liv-01_S38_L005_R1_001.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/Liv-01_S38_L005_R2_001.fastq.gz
!cutadapt -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -o ./fastq_trimmed/Liv_02_1.fastq.gz -p ./fastq_trimmed/Liv_02_2.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/Liv-02_S39_L005_R1_001.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/Liv-02_S39_L005_R2_001.fastq.gz
!cutadapt -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -o ./fastq_trimmed/Liv_03_1.fastq.gz -p ./fastq_trimmed/Liv_03_2.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/Liv-03_S40_L005_R1_001.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/Liv-03_S40_L005_R2_001.fastq.gz
!cutadapt -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -o ./fastq_trimmed/Liv_04_1.fastq.gz -p ./fastq_trimmed/Liv_04_2.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/Liv-04_S41_L005_R1_001.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/Liv-04_S41_L005_R2_001.fastq.gz

This is cutadapt 4.9 with Python 3.12.3
Command line parameters: -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -o ./fastq_trimmed/Int_04_1.fastq.gz -p ./fastq_trimmed/Int_04_2.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/Int-04_S45_L005_R1_001.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/Int-04_S45_L005_R2_001.fastq.gz
Processing paired-end reads on 1 core ...
Done           01:10:18   179,662,246 reads @  23.5 µs/read;   2.56 M reads/minute
Finished in 4218.643 s (23.481 µs/read; 2.56 M reads/minute).

=== Summary ===

Total read pairs processed:        179,662,246
  Read 1 with adapter:              55,990,071 (31.2%)
  Read 2 with adapter:              51,084,573 (28.4%)
Pairs written (passing filters):   179,662,246 (100.0%)

Total basepairs processed: 54,257,998,292 bp
  Read 1: 27,128,999,146 bp
  Read 2: 27,128,999,146 bp
Total written (filtered):  50,300,380,211 bp (9

In [2]:
!fastqc -t 16 fastq_trimmed/*.fastq.gz -o fastq_trimmed_qc

[0.001s][warning][os,container] Duplicate cpuset controllers detected. Picking /sys/fs/cgroup/cpuset, skipping /dev/cpuset.
application/gzip
application/gzip
Started analysis of Int_01_1.fastq.gz
application/gzip
application/gzip
Started analysis of Int_01_2.fastq.gz
application/gzip
application/gzip
application/gzip
application/gzip
Started analysis of Int_02_1.fastq.gz
Started analysis of Int_02_2.fastq.gz
Started analysis of Int_03_1.fastq.gz
Started analysis of Int_03_2.fastq.gz
Started analysis of Int_04_1.fastq.gz
Started analysis of Int_04_2.fastq.gz
Approx 5% complete for Int_01_1.fastq.gz
Approx 5% complete for Int_01_2.fastq.gz
Approx 10% complete for Int_01_1.fastq.gz
Approx 10% complete for Int_01_2.fastq.gz
Approx 15% complete for Int_01_2.fastq.gz
Approx 15% complete for Int_01_1.fastq.gz
Approx 20% complete for Int_01_2.fastq.gz
Approx 20% complete for Int_01_1.fastq.gz
Approx 25% complete for Int_01_2.fastq.gz
Approx 25% complete for Int_01_1.fastq.gz
Approx 5% complete

In [3]:
# !mkdir fastq_qc
!fastqc /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/*fastq.gz -o fastq_qc

[0.012s][warning][os,container] Duplicate cpuset controllers detected. Picking /sys/fs/cgroup/cpuset, skipping /dev/cpuset.
application/gzip
application/gzip
Started analysis of Int-01_S42_L005_R1_001.fastq.gz
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
Approx 5% complete for Int-01_S42_L005_R1_001.fastq.gz
Approx 10% complete for Int-01_S42_L005_R1_001.fastq.gz
Approx 15% complete for Int-01_S42_L005_R1_001.fastq.gz
Approx 20% complete for Int-01_S42_L005_R1_001.fastq.gz
Approx 25% complete for Int-01_S42_L005_R1_001.fastq.gz
Approx 30% complete for Int-01_S42_L005_R1_001.fastq.gz
Approx 35% complete for Int-01_S42_L005_R1_001.fastq.gz
Approx 40% complete for Int-01_S42_L005_R1_001.fastq.gz
Approx 45% complete for Int-01_S42_L005_R1_001.fastq.gz
Approx 50% complete for Int-01_S42_L005_R1_001.fa

In [4]:
# !mkdir fastq_trimmed_qc /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/multiqc_report.html
# !cp /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/multiqc_report.html multiqc_report.html
!multiqc -f -d fastq_qc/ fastq_trimmed_qc/ -n multiqc_report_all_ipynb.html


  /// ]8;id=304351;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.17

|           multiqc | Prepending directory to sample names
|           multiqc | Search path : /data/users/mccallke0364/Final_Project/Sm_Mira_IvT/fastq_qc
|           multiqc | Search path : /data/users/mccallke0364/Final_Project/Sm_Mira_IvT/fastq_trimmed_qc
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 22/22  stqc.html
|            fastqc | Found 11 reports
|           multiqc | Report      : multiqc_report_all_ipynb.html   (overwritten)
|           multiqc | Data        : multiqc_report_all_ipynb_data   (overwritten)
|           multiqc | MultiQC complete


In [1]:

# !mkdir alignment
# !mkdir alignment/star
# !mkdir alignment/hisat

In [5]:
!STAR \
--runThreadN 32 \
--runMode genomeGenerate \
--genomeDir genome_alignment/star \
--genomeFastaFiles genome/genome.fa \
--sjdbGTFfile genome/annotations.gtf \
--sjdbOverhang 150\
--genomeSAindexNbases 13

	STAR --runThreadN 32 --runMode genomeGenerate --genomeDir alignment/star --genomeFastaFiles genome/genome.fa --sjdbGTFfile genome/annotations.gtf --sjdbOverhang 150 --genomeSAindexNbases 13
	STAR version: 2.7.10b   compiled: 2022-11-01T09:53:26-04:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Dec 10 14:18:48 ..... started STAR run
Dec 10 14:18:48 ... starting to generate Genome files
Dec 10 14:18:54 ..... processing annotations GTF
Dec 10 14:18:57 ... starting to sort Suffix Array. This may take a long time...
Dec 10 14:18:59 ... sorting Suffix Array chunks and saving them to disk...
Dec 10 14:19:42 ... loading chunks from disk, packing SA...
Dec 10 14:19:52 ... finished generating suffix array
Dec 10 14:19:52 ... generating Suffix Array index
Dec 10 14:20:37 ... completed Suffix Array index
Dec 10 14:20:38 ..... inserting junctions into the genome indices
Dec 10 14:21:09 ... writing Genome to disk ...
Dec 10 14:21:10 ... writing Suffix Array to disk ...
Dec 10 14:21:14 ... wr

In [ ]:
!STAR \
--runThreadN 32 \
--runMode alignReads \
--genomeDir genome_alignment/star \
--readFilesManifest manifest.tsv \
--readFilesCommand zcat \
--outSAMtype BAM SortedByCoordinate \
--outSAMunmapped Within \
--outFileNamePrefix alignment/star/
!cp alignment/star/Log.final.out alignment/star/firstlog.final.out

In [9]:
!STAR \
--runThreadN 32 \
--runMode alignReads \
--genomeDir genome_alignment/star \
--readFilesManifest manifest.tsv \
--readFilesCommand zcat \
--outSAMtype BAM SortedByCoordinate \
--outSAMunmapped Within \
--outFileNamePrefix alignment/star/
--outSAMattributes NH HI AS nM RG \
--outFileNamePrefix alignment/star/ \
--sjdbFileChrStartEnd alignment/star/SJ.out.tab
!cp alignment/star/Log.final.out alignment/star/second.final.out

IndentationError: unexpected indent (845505616.py, line 2)

In [ ]:
# !while read -r line; do \
#     fq=$(echo $line | awk '{print $1}'); \
#     bn=$(basename "$fq" .fastq.gz); \
#     rg=$(echo $line | awk '{print $3}' | sed 's/ID://' ); \
#     echo "Aligning $fq with $rg"; \
#     hisat2 alignment/hisat/genome -p 16 -U $fq --rg-id $rg --rg SM:$rg --summary-file alignment/hisat/$bn.log --new-summary > alignment/hisat/$bn.sam; \
#     done < manifest.tsv

In [ ]:
# !for sam in alignment/hisat/*.sam; do \
#     bn=$(basename "$sam" .sam); \
#     echo "Converting $sam to $bn.bam and sorting."; \
#     samtools view -@ 32 -b $sam | samtools sort -@ 32 > alignment/hisat/$bn.bam; \
#     done

In [ ]:
# !samtools merge -@ 32 alignment/hisat/merged.bam  alignment/hisat/*.bam 
# !samtools index -@ 32 alignment/hisat/merged.bam 

In [ ]:
!samtools flagstat alignment/

In [ ]:
!qualimap bamqc -nt 32 -outdir qualimap/star/bam -bam alignment/star/Aligned.sortedByCoord.out.bam --feature-file genome/annotations.gtf 
!qualimap rnaseq -outdir qualimap/star/rnaseq -bam alignment/star/Aligned.sortedByCoord.out.bam -gtf genome/annotations.gtf
# !qualimap bamqc -nt 32 -outdir qualimap/hisat/bam -bam alignment/hisat/merged.bam --feature-file genome/annotations.gtf -p strand-specific-forward
# !qualimap rnaseq -outdir qualimap/hisat/rnaseq -bam alignment/hisat/merged.bam -gtf genome/annotations.gtf -p strand-specific-forward


In [ ]:
!multiqc --force -d fastq_qc/ fastq_trimmed_qc/ alignment/ .

In [ ]:
!mkdir logs
!mkdir dedup

In [ ]:

!picard MarkDuplicates -I alignment/star -M logs/star_duplicates -O dedup/star.bam --VALIDATION_STRINGENCY SILENT
# !picard MarkDuplicates -I alignment/hisat/merged.bam -M logs/hisat_duplicates -O dedup/hisat.bam --VALIDATION_STRINGENCY SILENT


In [ ]:
!featureCounts -T 32 \
    dedup/star.bam \
    -T 32 \
    --byReadGroup \
    -s 1 \
    --ignoreDup \
    -M \
    --fraction \
    -a genome/annotations.gtf \
    -o star_counts.tsv \
    --verbose


# !featureCounts -T 32 \
#     dedup/hisat.bam \
#     -T 32 \
#     --byReadGroup \
#     -s 1 \
#     --ignoreDup \
#     -M \
#     --fraction \
#     -a genome/annotations.gtf \
#     -o hisat_counts.tsv \
#     --verbose 

In [ ]:
!multiqc --force -d fastq_qc/ fastq_trimmed_qc/ alignment/ alignment_qc.

in single end seq we get a single end primer that does the seq by synthesis and seq all accross the fragment and reads the fluoresence
for paired end we add an entire second round that adds to the other end and does the reverse end of the original 
star deals with reads that overlap differently based on their number? maybe